In [1]:
import polars as pl
import numpy as np

In [2]:
data = pl.read_csv("data/comp_df.csv")

In [4]:
cat_category = ["hours","remote","education","seniority", "country", "sample_date"]
str_category = ["job_name", "all_tags"]
int_category = ["comp_dol"]

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.base import TransformerMixin
from sklearn.ensemble import HistGradientBoostingRegressor

In [6]:
col_transformer = make_column_transformer(
    (OneHotEncoder(handle_unknown='ignore'), cat_category),
    (CountVectorizer(stop_words="english") , "job_name"),
    (CountVectorizer(stop_words="english") , "all_tags")
       
)

In [7]:
col_transformer

ColumnTransformer(transformers=[('onehotencoder',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['hours', 'remote', 'education', 'seniority',
                                  'country', 'sample_date']),
                                ('countvectorizer-1',
                                 CountVectorizer(stop_words='english'),
                                 'job_name'),
                                ('countvectorizer-2',
                                 CountVectorizer(stop_words='english'),
                                 'all_tags')])

In [8]:
scaler = StandardScaler()

In [9]:
X = data.select(pl.col("hours","remote","education","seniority", "country", "sample_date","job_name", "all_tags"))
col_transformer.fit_transform(X).toarray().shape
y = data.select(pl.col("comp_dol"))
y_trans = scaler.fit_transform(y)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y_trans, test_size = 0.2, shuffle=True, random_state = 2137)

In [11]:
class DenseTransformer(TransformerMixin):

    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X, y=None, **fit_params):
        return np.asarray(X.todense())

In [12]:
hist_pipe = make_pipeline(
    col_transformer,
    DenseTransformer(),
    HistGradientBoostingRegressor()
)

In [13]:
hist_pipe.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['hours', 'remote',
                                                   'education', 'seniority',
                                                   'country', 'sample_date']),
                                                 ('countvectorizer-1',
                                                  CountVectorizer(stop_words='english'),
                                                  'job_name'),
                                                 ('countvectorizer-2',
                                                  CountVectorizer(stop_words='english'),
                                                  'all_tags')])),
                ('densetransformer',
                 <__main__.DenseTransformer object at 0x76c9d1e466e0>),
                ('histgradientboostingregressor',
                 HistGradientBoostingRegressor())])

In [14]:
hist_pred = hist_pipe.predict(X_test)

In [15]:
mse_hist = mean_squared_error(y_test, hist_pred)
r2_hist = r2_score(y_test, hist_pred)

In [16]:
print(f'Baseline Model Mean Squared Error:  {mse_hist}\n')
print(f'Baseline Model R2-score: {r2_hist}')

Baseline Model Mean Squared Error:  0.6614434538403823

Baseline Model R2-score: 0.387328130899558
